# Tutorial: Covariates

This notebook demonstrates covariate preparation, validation, and covariate-aware checks.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from pydantic import ValidationError

from tollama.core.registry import ModelCapabilities
from tollama.core.schemas import ForecastRequest
from tollama.daemon.covariates import (
    apply_covariate_capabilities,
    normalize_covariates,
)

In [ ]:
history = pd.DataFrame({
    'timestamp': pd.date_range('2025-01-01', periods=12, freq='D'),
    'target': [100, 104, 103, 107, 111, 116, 119, 121, 125, 129, 131, 136],
    'promo': [0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1],
    'price': [10.0, 9.8, 10.1, 9.9, 9.7, 10.0, 10.1, 9.6, 9.5, 9.7, 9.8, 9.4],
})
future_covariates = {'promo': [1, 1, 0], 'price': [9.3, 9.2, 9.2]}
history.head()

In [ ]:
payload = {
    'model': 'mock',
    'horizon': 3,
    'quantiles': [0.1, 0.5, 0.9],
    'series': [
        {
            'id': 'store_001',
            'freq': 'D',
            'timestamps': history['timestamp'].dt.strftime('%Y-%m-%d').tolist(),
            'target': history['target'].tolist(),
            'past_covariates': {
                'promo': history['promo'].tolist(),
                'price': history['price'].tolist(),
            },
            'future_covariates': future_covariates,
        }
    ],
    'parameters': {'covariates_mode': 'best_effort'},
    'options': {},
}
request = ForecastRequest.model_validate(payload)
normalized, normalize_warnings = normalize_covariates(request.series, request.horizon)
normalized[0].model_dump(exclude_none=True), normalize_warnings

In [ ]:
unsupported = ModelCapabilities()
_, best_effort_warnings = apply_covariate_capabilities(
    model_name='mock',
    model_family='mock',
    inputs=normalized,
    capabilities=unsupported,
    covariates_mode='best_effort',
)
print('Best-effort warnings:', len(best_effort_warnings))
print(best_effort_warnings[0])

try:
    apply_covariate_capabilities(
        model_name='mock',
        model_family='mock',
        inputs=normalized,
        capabilities=unsupported,
        covariates_mode='strict',
    )
except ValueError as exc:
    print('Strict-mode error:', exc)

In [ ]:
fig, ax1 = plt.subplots(figsize=(9, 3.8))
ax1.plot(history['timestamp'], history['target'], label='target', color='tab:blue')
ax1.set_ylabel('target')
ax2 = ax1.twinx()
ax2.step(history['timestamp'], history['promo'], where='mid', label='promo')
ax2.set_ylabel('promo')
ax1.set_title('Target and promo covariate')
ax1.grid(alpha=0.2)
plt.tight_layout()

In [ ]:
long_df = history.melt(
    id_vars='timestamp',
    value_vars=['target', 'promo', 'price'],
    var_name='series',
    value_name='value',
)
px.line(long_df, x='timestamp', y='value', color='series', title='Target and covariates')

In [ ]:
bad_payload = payload.copy()
bad_payload['series'] = [dict(payload['series'][0])]
bad_payload['series'][0]['future_covariates'] = {'promo': [1]}
try:
    ForecastRequest.model_validate(bad_payload)
except ValidationError as exc:
    print('Validation example:', exc.errors()[0]['msg'])